In [ ]:
import src.alex.reaction_diffusion as rd
import src.matijs.DLA as dla_matijs
import src.matijs.analysis as diffdla_analysis
import src.noa.metrics as metrics
import matplotlib.pyplot as plt 
import numpy as np
import src.noa.monte_carlo_DLA as dla
import pandas as pd

# Part I

In [ ]:
# Note this takes a few minutes to run, check results in report, not very interesting anyway
print("Optimal omegas for each eta in a 100x100 grid")
print("Eta | Omega | SOR Iterations")
for eta in [0.5,0.75,1,1.25,1.5,1.7]:
    omega, iters = diffdla_analysis.optimize_omega(100,100,eta)
    print(f"{eta} | {omega} | {iters}")

In [ ]:
diffdla_analysis.plot_end(N=100,M=100,eta=0.5,omega=1.77,growth_iterations=500)
diffdla_analysis.plot_end(N=100,M=100,eta=1,omega=1.78,growth_iterations=500)
diffdla_analysis.plot_end(N=100,M=100,eta=1.5,omega=1.9,growth_iterations=500)

In [ ]:
# fractal dimension calculation 
etas = np.linspace(0.1, 2.2, 22)
fractdims = []

for eta in etas:
    trial_fractdims = []
    for _ in range(5):
        material, _, _ = dla_matijs.dla(N=100, M=100, eta=eta, omega=1.77, growth_iterations=500)
        fractdim = metrics.fractal_dimension(material[-1])
        trial_fractdims.append(fractdim)
    avg_fractdim = np.mean(trial_fractdims)
    std_fractdim = np.std(trial_fractdims)
    fractdims.append((eta, avg_fractdim, std_fractdim))

# Convert to numpy array for easier indexing
fractdims = np.array(fractdims)

# Save to CSV
df = pd.DataFrame(fractdims, columns=['Eta', 'Average Fractal Dimension', 'Standard Deviation'])
df.to_csv('fig/fractal_dimensions.csv', index=False)

In [ ]:
# Load the data from CSV
df = pd.read_csv('fig/fractal_dimensions.csv')

# Plotting
plt.figure(dpi=300)
plt.errorbar(df['Eta'], df['Average Fractal Dimension'], yerr=df['Standard Deviation'], fmt='o', capsize=5)
plt.plot(df['Eta'], df['Average Fractal Dimension'])
plt.xlabel(r'$\eta$', fontsize=20)
plt.ylabel('Fractal Dimension', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.show()
plt.savefig('fig/dla_fractal_dimensions.png')

# Part II

In [ ]:
height, width = 100, 100
num_particles = 800
dla.plot_dla(height, width, num_particles)

In [ ]:
# sticking probabilities
ps = [0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
N = 25

# plot metrics to compare resulting clusters the sticking probabilities

dla.generate_cluster_data(height,width,num_particles,ps, N)
rgs, rgs_std, fractal_dims, fractal_dims_std = dla.data_analysis(ps,N)
dla.plot_data_analysis(ps, rgs, rgs_std, fractal_dims, fractal_dims_std)

# Part III

In [ ]:
# plot_html is for ipynb animation showing
rd.reset_global_UV()
rd.plot_html(boundary_condition="PBC", observed_variable = "V", f_value = 0.022, k_value = 0.051)

In [ ]:
# plot_html is for ipynb animation showing
rd.reset_global_UV()
rd.plot_html(boundary_condition="PBC", observed_variable = "V", f_value = 0.030, k_value = 0.060)

In [ ]:
# plot_html is for ipynb animation showing
rd.reset_global_UV()
rd.plot_html(boundary_condition="PBC", observed_variable = "V", f_value = 0.047, k_value = 0.063)

In [ ]:
# plot_animation is for saving the animation as a mp4 in the fig folder

# f = 0.022, k = 0.051
# f = 0.030, k = 0.060
# f = 0.047, k = 0.063
fkSet = [
    {0.022, 0.051},
    {0.030, 0.060},
    {0.047, 0.063}
    ]

for v in fkSet:
    f = v.pop()
    k = v.pop()
    
    # periodic boundary condition
    rd.plot_animation("PBC", observed_variable = "V", f_value = f, k_value = k)

    # Dirichlet boundary condition
    rd.plot_animation("Dirichlet Strong", observed_variable = "V", f_value = f, k_value = k)